In [ ]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from imutils.perspective import four_point_transform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from skimage.segmentation import clear_border
from tensorflow.keras.preprocessing.image import img_to_array
import sys
from copy import deepcopy

In [ ]:
#We Convert the image into B&W and apply blur to it after importing it.
 
im = cv2.imread('image.jpg')
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 3)
cv2.imshow(blurred)

In [ ]:
#Threshold filter and inversion is applied on the image
 
thresh = cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
thresh = cv2.bitwise_not(thresh)
cv2.imshow(thresh)

In [ ]:
#We grab the Contours from the image
 
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [ ]:
#Grab the Contour which has 4 defined vertexes
 
 
puzzleCnt = None
for c in cnts:
  peri = cv2.arcLength(c, True)
  approx = cv2.approxPolyDP(c, 0.02 * peri, True)
  if len(approx) == 4:
   puzzleCnt = approx
   break

In [ ]:
#Show the grabbed Contours
 
if puzzleCnt is None:
   print("Please upload another image, this one is skewed, too zoomed out or too zoomed in,not well lit")
 
else:
   output = im.copy()
   cv2.drawContours(output, [puzzleCnt],-1,(0, 255, 0), 2)
   cv2.imshow(output)

In [ ]:
#Get the bird's eye view of the Sudoku
 
puzzle = four_point_transform(im, puzzleCnt.reshape(4, 2))
warped = four_point_transform(gray, puzzleCnt.reshape(4, 2))
 
cv2.imshow(puzzle)

In [ ]:
#Get locations of each individual cell
 
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
 
loc = []
 
for i in range(0,9):
   tmp = []
   for j in range(0,9):
      x = (j*stepX,i*stepY,(j+1)*stepX,(i+1)*stepY)
      tmp.append(x)
   loc.append(tmp)

In [ ]:
#designing a CNN for prediction
 
classes,height,width,depth = 10,28,28,1
 
model = Sequential()
inputShape = (height, width, depth)
model.add(Conv2D(32, (5, 5), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes))
model.add(Activation("softmax"))

In [ ]:
#defining and training the CNN model
 
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
 
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
 
trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
testData = testData.reshape((testData.shape[0], 28, 28, 1))
 
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0
 
le = LabelBinarizer()
trainLabels = le.fit_transform(trainLabels)
testLabels = le.transform(testLabels)
 
opt = Adam(lr=0.001)
 
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
 
H = model.fit(trainData, trainLabels,validation_data=(testData, testLabels),batch_size=128,epochs=10,callbacks= [callback],verbose=1)

In [ ]:
#grabbing each cell and predicting it,if it's a number then predict it else output 0
 
grid = []
printgrid = []
 
for i in range(9):
   tmp = []
   for j in range(9):
 
      cell = warped[loc[i][j][1]:loc[i][j][3],loc[i][j][0]:loc[i][j][2]]
      thresh = cv2.threshold(cell, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
      thresh  = clear_border(thresh)
      cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
      cnts = imutils.grab_contours(cnts)
   
      if len(cnts) == 0:
        tmp.append(0)
        printgrid.append(loc[i][j])
      else:
        c = max(cnts, key=cv2.contourArea)
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        (h, w) = thresh.shape
        percentFilled = cv2.countNonZero(mask) / float(w * h)
 
        if percentFilled < 0.03:
          tmp.append(0)
          printgrid.append(loc[i][j])
        else:
          digit = cv2.bitwise_and(thresh, thresh, mask=mask)
          roi = cv2.resize(digit,(28, 28))
          roi = roi.astype("float") / 255.0
          roi = img_to_array(roi)
          roi = np.expand_dims(roi, axis=0)
          tmp.append(model.predict(roi).argmax())
   
   grid.append(tmp)

In [ ]:
ans = []
for i in range(9):
   for j in range(9):
      if grid[i][j] == 0:
         ans.append((i,j))

In [ ]:
#algorithm to solve sudoku
 
def output(a):
    sys.stdout.write(str(a))
 
N = 9
 
field = grid
 
def print_field(field):
    if not field:
        output("No solution")
        return
    for i in range(N):
        for j in range(N):
            cell = field[i][j]
            grid[i][j] = cell
            if cell == 0 or isinstance(cell, set):
                output('.')
            else:
                output(cell)
            if (j + 1) % 3 == 0 and j < 8:
                output(' |')
 
            if j != 8:
                output(' ')
        output('\n')
        if (i + 1) % 3 == 0 and i < 8:
            output("- - - + - - - + - - -\n")
 
def read(field):
    """ Read field into state (replace 0 with set of possible values) """
 
    state = deepcopy(field)
    for i in range(N):
        for j in range(N):
            cell = state[i][j]
            if cell == 0:
                state[i][j] = set(range(1,10))
 
    return state
 
state = read(field)
 
 
def done(state):
    """ Are we done? """
 
    for row in state:
        for cell in row:
            if isinstance(cell, set):
                return False
    return True
 
 
def propagate_step(state):
    """
    Propagate one step.
 
    @return:  A two-tuple that says whether the configuration
              is solvable and whether the propagation changed
              the state.
    """
 
    new_units = False
 
    # propagate row rule
    for i in range(N):
        row = state[i]
        values = set([x for x in row if not isinstance(x, set)])
        for j in range(N):
            if isinstance(state[i][j], set):
                state[i][j] -= values
                if len(state[i][j]) == 1:
                    val = state[i][j].pop()
                    state[i][j] = val
                    values.add(val)
                    new_units = True
                elif len(state[i][j]) == 0:
                    return False, None
 
    # propagate column rule
    for j in range(N):
        column = [state[x][j] for x in range(N)]
        values = set([x for x in column if not isinstance(x, set)])
        for i in range(N):
            if isinstance(state[i][j], set):
                state[i][j] -= values
                if len(state[i][j]) == 1:
                    val = state[i][j].pop()
                    state[i][j] = val
                    values.add(val)
                    new_units = True
                elif len(state[i][j]) == 0:
                    return False, None
 
    # propagate cell rule
    for x in range(3):
        for y in range(3):
            values = set()
            for i in range(3 * x, 3 * x + 3):
                for j in range(3 * y, 3 * y + 3):
                    cell = state[i][j]
                    if not isinstance(cell, set):
                        values.add(cell)
            for i in range(3 * x, 3 * x + 3):
                for j in range(3 * y, 3 * y + 3):
                    if isinstance(state[i][j], set):
                        state[i][j] -= values
                        if len(state[i][j]) == 1:
                            val = state[i][j].pop()
                            state[i][j] = val
                            values.add(val)
                            new_units = True
                        elif len(state[i][j]) == 0:
                            return False, None
 
    return True, new_units
 
def propagate(state):
    """ Propagate until we reach a fixpoint """
    while True:
        solvable, new_unit = propagate_step(state)
        if not solvable:
            return False
        if not new_unit:
            return True
 
 
def solve(state):
    """ Solve sudoku """
 
    solvable = propagate(state)
 
    if not solvable:
        return None
 
    if done(state):
        return state
 
    for i in range(N):
        for j in range(N):
            cell = state[i][j]
            if isinstance(cell, set):
                for value in cell:
                    new_state = deepcopy(state)
                    new_state[i][j] = value
                    solved = solve(new_state)
                    if solved is not None:
                        return solved
                return None

In [ ]:
#Show the Solved Sudoku

print_field(solve(state))

[[0, 5, 0, 0, 8, 0, 0, 1, 6],
 [2, 0, 0, 0, 0, 0, 0, 0, 5],
 [0, 0, 7, 0, 0, 7, 0, 0, 0],
 [5, 0, 0, 2, 0, 0, 0, 6, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 2, 0, 0, 0],
 [0, 0, 0, 7, 0, 0, 3, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 0, 0, 0, 1, 6, 0, 7, 0]]

In [ ]:
puz = puzzle.copy()

In [ ]:
#Visualize the sudoku
 
k = 0
for i in printgrid:
      if b >= 9:
         b = 0
         a += 1
      textX = int((i[2] - i[0]) * 0.33)
      textY = int((i[3] - i[1]) * -0.2)
      textX += i[0]
      textY += i[3]
      x = ans[k][0]
      y = ans[k][1]
      cv2.putText(puz, str(grid[x][y]), (textX, textY),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2)
      k += 1
  
cv2.imshow(puz)